In [0]:
dbutils.secrets.listScopes()

[SecretScope(name='cus360secret-id'),
 SecretScope(name='CUSTOMER360'),
 SecretScope(name='customer360_scope')]

In [0]:
dbutils.secrets.list("CUSTOMER360")
dbutils.secrets.get(scope="CUSTOMER360", key="storage-keyid")

'[REDACTED]'

# ESTABLISHING CONNECTION WITH ADLS GEN2

In [0]:
storage_account_name = "akashcustomer360"
access_key = "g4A/5+H1dANpSJSu7Jn/xsbphokBhUPEWGl7IQgVdYytgXXXACnCaiGNtkKImQ78+J0xsLYdgEmo+AStX9y54A=="  # Paste the new key here
spark.conf.set(
    f"fs.azure.account.key.{storage_account_name}.dfs.core.windows.net", 
    access_key
)

In [0]:
customer_master_path = "abfss://data@akashcustomer360.dfs.core.windows.net/source/customer_master.xlsx"
customer_support_path = "abfss://data@akashcustomer360.dfs.core.windows.net/source/customer_support_tickets.xlsx"
transactions_path = "abfss://data@akashcustomer360.dfs.core.windows.net/source/transactions.xlsx"
web_interactions_path = "abfss://data@akashcustomer360.dfs.core.windows.net/source/web_interactions.xlsx"

# READING XLSX FILES

In [0]:
import pandas as pd
from adlfs import AzureBlobFileSystem
from pyspark.sql.functions import current_timestamp, lit

# Create filesystem instance
fs = AzureBlobFileSystem(
    account_name=storage_account_name,
    account_key=access_key
)

# Read the Excel file
with fs.open(customer_master_path, mode="rb") as f:
    customer_master_df = pd.read_excel(f, engine="openpyxl")

# Process the date column
customer_master_df['signup_date'] = pd.to_datetime(
    customer_master_df['signup_date'],
    origin='1899-12-30',
    unit='D',
    errors='coerce'
)


# Read customer_support.xlsx
with fs.open(customer_support_path, mode="rb") as f:
    customer_support_df = pd.read_excel(f, engine="openpyxl")

# Read the file
with fs.open(web_interactions_path, mode="rb") as f:
    web_interactions_df = pd.read_excel(f, engine="openpyxl")

web_interactions_df['interaction_date'] = pd.to_datetime(
    web_interactions_df['interaction_date'],
    origin='1899-12-30',  # Excel's epoch (Windows)
    unit='D',             # Days since epoch
    errors='coerce'      # Invalid dates → NaT
)

# Read the file
with fs.open(transactions_path, mode="rb") as f:
    transactions_df = pd.read_excel(f, engine="openpyxl")

# Convert Excel dates to datetime
transactions_df['transaction_date'] = pd.to_datetime(
    transactions_df['transaction_date'],
    origin='1899-12-30',  # Excel's epoch (Windows)
    unit='D',             # Days since epoch
    errors='coerce'      
)





In [0]:

customer_master_df['city'] = customer_master_df['location'].str.extract(r'^([^,]+),\s*[A-Z]{2}$').fillna('Unknown')
customer_master_df['state_code'] = customer_master_df['location'].str.extract(r',\s*([A-Z]{2})$').fillna('Unknown')
customer_master_df.drop('location', axis=1)


,customer_id,name,email,phone,signup_date,city,state_code
0,C0001,Henry Martinez,henry.martinez@example.com,555-520-5205,2023-09-01,New York,NY
1,C0002,Alice Garcia,alice.garcia@example.com,555-635-6063,2023-06-11,Boston,MA
2,C0003,Carol Wilson,carol.wilson@example.com,555-178-8465,2023-01-17,Boston,MA
3,C0004,Carol Johnson,carol.johnson@example.com,555-145-4221,2023-05-15,Boston,MA
4,C0005,Frank Johnson,frank.johnson@example.com,555-272-7357,2023-10-07,New York,NY
...,...,...,...,...,...,...,...
995,C0996,Grace Lee,grace.lee@example.com,555-591-1894,2023-10-26,San Diego,CA
996,C0997,David Taylor,david.taylor@example.com,555-870-8476,2023-05-02,Chicago,IL
997,C0998,Grace Martinez,grace.martinez@example.com,555-421-1063,2023-03-24,San Diego,CA
998,C0999,Jack Wilson,jack.wilson@example.com,555-454-5688,2023-01-14,Seattle,WA


In [0]:
customer_support_df

,ticket_id,customer_id,created_at,resolved_at,issue_type,satisfaction_score
0,TCKT1000,C0639,2023-09-15,2023-09-21,Login Issue,5
1,TCKT1001,C0538,2023-06-22,2023-06-29,Login Issue,3
2,TCKT1002,C0254,2023-06-11,2023-06-12,Account Closure,1
3,TCKT1003,C0504,2023-09-11,2023-09-17,Account Closure,1
4,TCKT1004,C0945,2023-04-12,2023-04-16,Payment Problem,2
...,...,...,...,...,...,...
995,TCKT1995,C0553,2023-11-01,2023-11-08,Account Closure,4
996,TCKT1996,C0267,2023-06-23,2023-06-27,Account Closure,3
997,TCKT1997,C0803,2023-08-27,2023-08-30,Technical Bug,5
998,TCKT1998,C0590,2023-03-05,2023-03-10,Order Delay,2


In [0]:
display(customer_master_df)

customer_id,name,email,phone,signup_date,location,city,state_code
C0001,Henry Martinez,henry.martinez@example.com,555-520-5205,2023-09-01T00:00:00Z,"New York, NY",New York,NY
C0002,Alice Garcia,alice.garcia@example.com,555-635-6063,2023-06-11T00:00:00Z,"Boston, MA",Boston,MA
C0003,Carol Wilson,carol.wilson@example.com,555-178-8465,2023-01-17T00:00:00Z,"Boston, MA",Boston,MA
C0004,Carol Johnson,carol.johnson@example.com,555-145-4221,2023-05-15T00:00:00Z,"Boston, MA",Boston,MA
C0005,Frank Johnson,frank.johnson@example.com,555-272-7357,2023-10-07T00:00:00Z,"New York, NY",New York,NY
C0006,Jack Brown,jack.brown@example.com,555-339-4433,2023-01-12T00:00:00Z,"Phoenix, AZ",Phoenix,AZ
C0007,Alice Brown,alice.brown@example.com,555-901-6057,2023-09-21T00:00:00Z,"San Diego, CA",San Diego,CA
C0008,Emma Brown,emma.brown@example.com,555-723-4966,2023-02-19T00:00:00Z,"Denver, CO",Denver,CO
C0009,Jack Miller,jack.miller@example.com,555-682-9443,2023-09-26T00:00:00Z,"San Diego, CA",San Diego,CA
C0010,Emma Martinez,emma.martinez@example.com,555-291-5870,2023-12-13T00:00:00Z,"San Diego, CA",San Diego,CA


In [0]:
display(customer_support_df)

ticket_id,customer_id,created_at,resolved_at,issue_type,satisfaction_score
TCKT1000,C0639,2023-09-15T00:00:00Z,2023-09-21T00:00:00Z,Login Issue,5
TCKT1001,C0538,2023-06-22T00:00:00Z,2023-06-29T00:00:00Z,Login Issue,3
TCKT1002,C0254,2023-06-11T00:00:00Z,2023-06-12T00:00:00Z,Account Closure,1
TCKT1003,C0504,2023-09-11T00:00:00Z,2023-09-17T00:00:00Z,Account Closure,1
TCKT1004,C0945,2023-04-12T00:00:00Z,2023-04-16T00:00:00Z,Payment Problem,2
TCKT1005,C0686,2023-04-05T00:00:00Z,2023-04-06T00:00:00Z,Technical Bug,2
TCKT1006,C0471,2023-02-13T00:00:00Z,2023-02-18T00:00:00Z,Login Issue,2
TCKT1007,C0723,2023-03-20T00:00:00Z,2023-03-23T00:00:00Z,Account Closure,1
TCKT1008,C0693,2023-08-17T00:00:00Z,2023-08-24T00:00:00Z,Payment Problem,4
TCKT1009,C0689,2023-11-17T00:00:00Z,2023-11-23T00:00:00Z,Account Closure,4


In [0]:
transactions_df['city'] = transactions_df['store_location'].str.extract(r'^([^,]+),\s*[A-Z]{2}$').fillna('Unknown')
transactions_df['state_code'] = transactions_df['store_location'].str.extract(r',\s*([A-Z]{2})$').fillna('Unknown')
transactions_df.drop('store_location', axis=1)

,transaction_id,customer_id,transaction_date,amount,payment_method,city,state_code
0,T10000,C0423,2023-07-10,323.19,Cash,Seattle,WA
1,T10001,C0944,2023-11-27,155.72,Credit Card,New York,NY
2,T10002,C0090,2023-08-05,33.93,PayPal,Austin,TX
3,T10003,C0950,2023-10-18,403.96,Debit Card,Chicago,IL
4,T10004,C0106,2023-05-03,241.11,Cash,New York,NY
...,...,...,...,...,...,...,...
995,T10995,C0261,2023-03-14,429.59,Cash,Seattle,WA
996,T10996,C0638,2023-05-20,476.92,PayPal,Chicago,IL
997,T10997,C0237,2023-11-26,319.34,Debit Card,Seattle,WA
998,T10998,C0717,2023-07-15,39.73,PayPal,Miami,FL


In [0]:
display(transactions_df)

transaction_id,customer_id,transaction_date,amount,payment_method,store_location,city,state_code
T10000,C0423,2023-07-10T00:00:00Z,323.19,Cash,"Seattle, WA",Seattle,WA
T10001,C0944,2023-11-27T00:00:00Z,155.72,Credit Card,"New York, NY",New York,NY
T10002,C0090,2023-08-05T00:00:00Z,33.93,PayPal,"Austin, TX",Austin,TX
T10003,C0950,2023-10-18T00:00:00Z,403.96,Debit Card,"Chicago, IL",Chicago,IL
T10004,C0106,2023-05-03T00:00:00Z,241.11,Cash,"New York, NY",New York,NY
T10005,C0726,2023-10-21T00:00:00Z,432.72,PayPal,"New York, NY",New York,NY
T10006,C0382,2023-09-14T00:00:00Z,70.84,Credit Card,"Chicago, IL",Chicago,IL
T10007,C0110,2023-01-05T00:00:00Z,128.5,Debit Card,"Austin, TX",Austin,TX
T10008,C0295,2023-08-12T00:00:00Z,147.3,Cash,"New York, NY",New York,NY
T10009,C0544,2023-04-04T00:00:00Z,168.88,Cash,"Chicago, IL",Chicago,IL


In [0]:
display(web_interactions_df)

session_id,customer_id,page_views,session_duration,device,interaction_date
S1000,C0059,10,95,Desktop,2023-11-09T00:00:00Z
S1001,C0638,8,6,Mobile,2023-07-09T00:00:00Z
S1002,C0790,15,60,Tablet,2023-08-08T00:00:00Z
S1003,C0527,12,47,Tablet,2023-03-13T00:00:00Z
S1004,C0375,14,53,Tablet,2023-08-20T00:00:00Z
S1005,C0171,7,179,Tablet,2023-10-08T00:00:00Z
S1006,C0474,5,165,Tablet,2023-05-26T00:00:00Z
S1007,C0281,13,9,Desktop,2023-04-25T00:00:00Z
S1008,C0277,7,162,Desktop,2023-02-17T00:00:00Z
S1009,C0053,19,114,Tablet,2023-09-08T00:00:00Z


# SNOWFLAKE CONFIGURATIONS

In [0]:
# snowflake_config= {
#   "sfURL": "ABIQMGE-XO81242.snowflakecomputing.com",
#   "sfUser": "AKASH1204",
#   "sfPassword": "Sai_Akash1204@2004",
#   "sfDatabase": "CUSTOMER360",
#   "sfSchema": "BRONZE_LAYER",
#   "sfWarehouse": "CUSTOMER360",
#   "sfRole": "ACCOUNTADMIN",
#   # internal stage
# }

In [0]:

# snowflake_config["sfFileFormat"] = "csv" 

# CONVERTING PANDAS DF TO SPARK DF

In [0]:
customer_master_spark_df = spark.createDataFrame(customer_master_df)

In [0]:
customer_support_spark_df = spark.createDataFrame(customer_support_df)

In [0]:
transactions_spark_df = spark.createDataFrame(transactions_df)

In [0]:
web_interactions_spark_df = spark.createDataFrame(web_interactions_df)

# INCLUDING AUDIT COLUMNS IN THE DATAFRAMES

In [0]:
customer_master_spark_df = customer_master_spark_df \
    .withColumn("ingested_at", current_timestamp()) \
    .withColumn("updated_at", current_timestamp()) \
    .withColumn("created_by", lit("DATABRICKS_INGEST_PIPELINE")) \
    .withColumn("updated_by", lit("DATABRICKS_INGEST_PIPELINE"))


In [0]:

customer_support_spark_df = customer_support_spark_df \
    .withColumn("ingested_at", current_timestamp()) \
    .withColumn("updated_at", current_timestamp()) \
    .withColumn("created_by", lit("DATABRICKS_INGEST_PIPELINE")) \
    .withColumn("updated_by", lit("DATABRICKS_INGEST_PIPELINE"))

In [0]:
transactions_spark_df = transactions_spark_df \
    .withColumn("ingested_at", current_timestamp()) \
    .withColumn("updated_at", current_timestamp()) \
    .withColumn("created_by", lit("DATABRICKS_INGEST_PIPELINE")) \
    .withColumn("updated_by", lit("DATABRICKS_INGEST_PIPELINE"))


In [0]:
web_interactions_spark_df = web_interactions_spark_df \
    .withColumn("ingested_at", current_timestamp()) \
    .withColumn("updated_at", current_timestamp()) \
    .withColumn("created_by", lit("DATABRICKS_INGEST_PIPELINE")) \
    .withColumn("updated_by", lit("DATABRICKS_INGEST_PIPELINE"))

# DUMPING INTO ADLS AS DELTA FORMAT


In [0]:
storage_account_name = "bronzecontainer"
access_key = "aS7Sy99bSL4cIvnwb8igw6wOrb+dxEM2dQ+JKB5z/38fZ32QTXfXl1TZBu+GJyyySDshs9n2scle+AStrYclsQ=="  # Paste the new key here
spark.conf.set(
    f"fs.azure.account.key.{storage_account_name}.dfs.core.windows.net", 
    access_key
)

In [0]:
customer_master_spark_df.write \
    .format("delta") \
    .option("mergeSchema", "true") \
    .mode("append") \
    .save("abfss://bronze-dump@bronzecontainer.dfs.core.windows.net/customer_master_raw")

In [0]:

    # CUSTOMER_SUPPORT
customer_support_spark_df.write \
    .format("delta") \
    .option("mergeSchema", "true") \
    .mode("append") \
    .save("abfss://bronze-dump@bronzecontainer.dfs.core.windows.net/customer_support_raw")


In [0]:
# TRANSACTIONS
transactions_spark_df.write \
    .format("delta") \
    .option("mergeSchema", "true") \
    .mode("append") \
    .save("abfss://bronze-dump@bronzecontainer.dfs.core.windows.net/transactions_raw")

In [0]:
# WEB INTERACTIONS
web_interactions_spark_df.write \
    .format("delta") \
    .option("mergeSchema", "true") \
    .mode("append") \
    .save("abfss://bronze-dump@bronzecontainer.dfs.core.windows.net/web_interactions_raw")

